# Information Robust Dirichlet Networks

In this notebook, we implement the specialized training loss proposed in the paper _Information Robust Dirichlet Networks for Predictive Uncertainty Estimation_ by Tsiligkaridis (2019). The method models predictive uncertainty by having a neural network output Dirichlet concentration parameters 𝛼 instead of just a pointwise softmax.

The total loss is composed of three terms:

1. Calibration term: implemented in the function  lp_fn
2. Regularization term: implemented in the function  regularization_fn
3. Adversiarial Entropy penalty: implemented in the function  dirichlet_entropy

In the paper and in this notenbook, L_p loss is not directly computed but rather an upper bound for it, denoted by F_i (for sample i)  

The regularization term penalizes high alpha values for incorrect classes.  

The final term uses the alpha values the model assigns to adversarial inputs.
The model is rewarded for outputting a Dirichlet-distribution with high entropy on these inputs.


In [33]:
import torch
from torch.special import digamma

def lp_fn(alpha, y, p):
    """
    Computes F_i (one per sample) using the expectation-based formulation:
        F_i = ( E[(1-p_c)^p] + Σ_{j≠c} E[p_j^p] )^(1/p)
        Then sums all F_i 
    Args:
        alpha : (B, K) Dirichlet concentration parameters (>0)
        y     : (B, K) one-hot labels
        p     : scalar exponent

    Returns:
        Σ_i F_i : Float
    """

    B, K = alpha.shape

    # total concentration α0
    alpha0 = alpha.sum(dim=1, keepdim=True)  # (B,1)

    # extract α_c (correct class)
    alpha_c = (alpha * y).sum(dim=1, keepdim=True)  # (B,1)
    alpha0_minus_c = alpha0 - alpha_c               # (B,1)

    # log B(a,b) used for expectations: E[X^p] = B(a+p,b)/B(a,b)
    def logB(a, b):
        return torch.lgamma(a) + torch.lgamma(b) - torch.lgamma(a + b)

    # E[(1 - p_c)^p]   where (1 - p_c) ~ Beta( α0 - α_c , α_c )
    log_E1 = logB(alpha0_minus_c + p, alpha_c) - logB(alpha0_minus_c, alpha_c)
    E1 = torch.exp(log_E1)  # (B,1)

    # Per-class E[p_j^p] for all j
    log_Ep = logB(alpha + p, alpha0 - alpha) - logB(alpha, alpha0 - alpha)  # (B,K)
    Ep = torch.exp(log_Ep)

    # zero-out the true class term so we sum only j≠c
    Ep = Ep * (1 - y)

    # final expectation sum
    E_sum = E1 + Ep.sum(dim=1, keepdim=True)  # (B,1)

    # apply ^(1/p)
    Fi = E_sum.pow(1.0 / p).squeeze(1)  # (B,)

    return Fi.sum()


def regularization_fn(alpha, y):
    """
    Implements the regularization term using trigamma functions and an equivalent formulation of the regularization term.

    alpha: (B, K)   Dirichlet parameters > 0
    y:     (B, K)   one-hot labels
    
        returns:
            Float
    """

    # Build alpha_tilde by replacing correct-class alpha with 1
    alpha_tilde = alpha * (1 - y) + y

    # Compute alpha_tilde_0 = 1 + sum over incorrect classes
    alpha_tilde_0 = torch.sum(alpha_tilde, dim=1, keepdim=True)

    # Polygamma(1, x) = trigamma(x)
    trigamma_alpha = torch.polygamma(1, alpha_tilde)
    trigamma_alpha0 = torch.polygamma(1, alpha_tilde_0)

    # (alpha_tilde - 1)^2 term
    diff_sq = (alpha_tilde - 1.0)**2

    # Penalty only for incorrect classes → mask out true class
    mask = (1 - y)

    # Compute elementwise contribution
    term = 0.5 * diff_sq * (trigamma_alpha - trigamma_alpha0) * mask

    # Sum over classes and batch
    return torch.sum(torch.sum(term, dim=1))


def dirichlet_entropy(alpha):
    """
    alpha: tensor of shape (B_a, K), where B_a = number of adversarial inputs
    returns: entropy : Float
    """
    K = alpha.size(-1)
    alpha0 = alpha.sum(dim=-1)

    log_B = torch.lgamma(alpha).sum(dim=-1) - torch.lgamma(alpha0)

    term1 = log_B
    term2 = (alpha0 - K) * digamma(alpha0)
    term3 = ((alpha - 1) * digamma(alpha)).sum(dim=-1)
    entropy = term1 + term2 - term3

    return entropy.sum()

def loss_IRD(alpha, y, adversarial_alpha = None, p = 2, lam = 1.0, gamma = 1.0):
    """
    Computes the Loss introduced in paper: Information Robust Dirichlet Networks for Predictive Uncertainty Estimation
    Args:
        alpha : (B, K) Dirichlet concentration parameters
        adversarial_alpha : (B_a, K) adversarial_alpha concentration parameters for adversarial inputs
        y     : (B, K) one-hot labels 
        p     : scalar exponent
    Returns:
        loss_IRD : the IRD loss comprised of all three terms, summed over all input examples
    """
    lp_term = lp_fn(alpha, y, p)
    reg_term = regularization_fn(alpha, y)
    if adversarial_alpha is not None:
        entropy_term = dirichlet_entropy(adversarial_alpha)
    else:
        entropy_term = 0.0
        
    return lp_term + lam * reg_term - gamma * entropy_term
    
    
    

In [34]:
# small test
alpha = torch.tensor([[2.5, 1.2, 0.8],
                      [1.2, 4.0, 1.3]], requires_grad=True)

adversarial_alpha = torch.tensor([[5, 4, 3],
                      [1.2, 2.0, 1.3]], requires_grad=True)

y = torch.tensor([[1,0,0],
                  [0,1,0]], dtype=torch.float32)

loss = loss_IRD(alpha, y, adversarial_alpha)
print(loss)            # tensor of shape (B,)
loss.mean().backward() # gradients flow properly

tensor(3.6637, grad_fn=<SubBackward0>)
